In [1]:
import pandas as pd

# CSV 파일 읽기
money = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(추정매출-상권).csv')  # DataFrame 객체로 로드
human = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(길단위인구-상권).csv')
map_sang = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(영역-상권).csv')
coordinate = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\위도경도.csv')

In [2]:
#필요한 컬럼만 선택하기
money = money[['기준_년분기_코드','상권_구분_코드_명','상권_코드','상권_코드_명','당월_매출_금액','남성_매출_금액','여성_매출_금액','서비스_업종_코드_명']]
human = human[['기준_년분기_코드','상권_코드','총_유동인구_수','남성_유동인구_수','여성_유동인구_수']]
map_sang = map_sang[['상권_코드','자치구_코드_명','행정동_코드_명']]
coordinate = coordinate[['상권_코드','경도1','위도1']]

In [3]:
# 추정매출과 길단위인구 조인 1step
join_1step = pd.merge(money, human, on=['기준_년분기_코드', '상권_코드'], how='inner')

In [4]:
# 1차에서 조인 되지 못한 상권코드
unmatched_from_df1 = money.merge(human, on=['기준_년분기_코드', '상권_코드'], how='left', indicator=True)
unmatched_code = unmatched_from_df1[unmatched_from_df1['_merge'] == 'left_only']

In [5]:
# 1step에서 조인 한것과 영역 데이터 조인(2차조인)
join_2step = pd.merge(join_1step, map_sang, on=['상권_코드'], how='inner')

In [6]:
#2차에서 조인 되지 못한 상권코드
unmatched_from_df1 = join_1step.merge(map_sang, on=['상권_코드'], how='left', indicator=True)
unmatched_code2 = unmatched_from_df1[unmatched_from_df1['_merge'] == 'left_only']

In [7]:
# '기준_년분기_코드' 값을 년월로 변환하는 함수 정의
def convert_to_date(code):
    year = int(str(code)[:4])  # 연도 추출
    quarter = int(str(code)[-1])  # 분기 추출
    month = (quarter - 1) * 3 + 1  # 분기에 따라 시작 월 계산
    return pd.Timestamp(year=year, month=month, day=1)
# 변환 적용
join_2step['기준_년분기_코드_날짜'] = join_2step['기준_년분기_코드'].apply(convert_to_date)

In [8]:
join_3step = join_2step[join_2step['기준_년분기_코드_날짜'] >= pd.Timestamp(year=2022, month=1, day=1)]


In [9]:
# 데이터 타입 최적화
coordinate['상권_코드'] = coordinate['상권_코드'].astype('category')
join_3step['상권_코드'] = join_3step['상권_코드'].astype('category')


C:\Users\m\AppData\Local\Temp\ipykernel_6564\1505405958.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  join_3step['상권_코드'] = join_3step['상권_코드'].astype('category')


In [10]:
# 2step에서 조인 한것과 영역 데이터 조인(3차조인)
join_final = pd.merge(join_3step, coordinate, on=['상권_코드'], how='inner')

In [11]:
#3차에서 조인 되지 못한 상권코드
unmatched_from_df2 = join_3step.merge(coordinate, on=['상권_코드'], how='left', indicator=True)
unmatched_code3 = unmatched_from_df2[unmatched_from_df2['_merge'] == 'left_only']

In [12]:
# 중복 제거
join_final = join_final.drop_duplicates()

In [13]:
#최종 데이터 저장
join_final.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\종합_대시보드데이터.csv', index=False, encoding='utf-8-sig')
unmatched_code.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\유동인구없는_상권코드.csv', index=False, encoding='utf-8-sig')
unmatched_code3.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\좌표없는_상권코드.csv', index=False, encoding='utf-8-sig')